Check Bash Version
------------------------------

In [ ]:
echo $BASH_VERSION

Variables
--------------

In [1]:
myname="Hans Schmid"
echo "Hello, $myname"

Hello, Hans Schmid


: 1

In [3]:
echo $myname

Hans Schmid


: 1

String interpolation

In [4]:
echo "$myname"

Hans Schmid


: 1

Literal string

In [5]:
echo '$MYNAME'

$MYNAME


: 1

In [6]:
type pwd

pwd is a shell builtin


: 1

In [7]:
echo $(pwd)

/home/schmidh/Gitrepos/bash-tutorial


: 1

In [8]:
echo $PWD

/home/schmidh/Gitrepos/bash-tutorial


: 1

Backticks are regarded as obsolete. But hard to kill in the field.

In [10]:
echo `pwd`

/home/schmidh/Gitrepos/bash-tutorial


: 1

Keywords
---------------
Braces, double brackets and exclamation mark are keywords. That means in order to use them correctly you need spaces around them!

In [11]:
type if then elif else fi time for in until while do done case esac coproc select function { } [[ ]] !

if is a shell keyword
then is a shell keyword
elif is a shell keyword
else is a shell keyword
fi is a shell keyword
time is a shell keyword
for is a shell keyword
in is a shell keyword
until is a shell keyword
while is a shell keyword
do is a shell keyword
done is a shell keyword
case is a shell keyword
esac is a shell keyword
coproc is a shell keyword
select is a shell keyword
function is a shell keyword
{ is a shell keyword
} is a shell keyword
[[ is a shell keyword
]] is a shell keyword
! is a shell keyword


: 1

Equal sign is not a keyword. So when using it you do not put spaces around it.

In [12]:
type =

bash: type: =: not found


: 1

which is obsolete. You'll find it a lot though.
<br>
**'which' is a bitch.**

In [13]:
type which

which is /usr/bin/which


: 1

Use **type -p** instead because it's a builtin!

In [14]:
type -p which

/usr/bin/which


: 1

In [15]:
type type

type is a shell builtin


: 1

# Functions

1. Functions in Bash are not functions like in other languages. They are actually commands. Functions are used as if they were command line binaries or scripts.
2. Shell commands are connected by pipes (aka streams), and not by fundamental or user-defined data types as in _real_ programming languages. There is no such thing as a return value for a command. (Although there is a return statement - see below.)
3. When a function wants to get input it reads it from its input stream, or the argument list. In both cases text strings have to be parsed. The shell with its pipe-based architecture is purely string-based!
4. When a command wants to return something it has to _echo/printf_ it to its output stream.
5. The caller can use command substitution _$()_ to capture the output.
6. Another practiced way is to store the return values in dedicated, global variables. (Writing to the output stream is clearer and more flexible, because it can also take binary data.)
7. Think of the exit code as a bonus that other languages don't have, or as a "Schmutzeffekt" of shell functions. The meaning of the exit code is up to the shell programmer!
8. Often by convention an exit status of 0 means success, 1-255 means failure and the exit code can be regarded as a failure number.
9. Just to make sure: _return/exit_ can only take a value from 0-255. Values other than 0 are _not necessarily_ errors. There is always an exit status.
10. The argument list is only for decoration and you never put anything inside them.

## How to define functions
There are two ways to define functions.

### 1. With the **_function_** keyword:
a) The argument list is optional.<br>
b) Coding style: Do not use an argument list.

In [16]:
function success {
    echo "I am successful!"
    return 0
}

: 1

### 2. Without the **_function_** keyword.
The argument list is mandatory.

In [17]:
success() {
    echo "I am successful!"
    return 0
}

: 1

### Passing and Accessing Arguments

In [18]:
function printme {
    echo "You gave me $# argument(s)!"
    echo -n $1
}

: 1

In [19]:
printme

You gave me 0 argument(s)!


: 1

In [20]:
printme 'Hello'

You gave me 1 argument(s)!
Hello

: 1

In [21]:
printme 'Hello' 'World'

You gave me 2 argument(s)!
Hello

: 1

### Returning Values

#### 1. echo/printf values to the output stream

In [23]:
function add {
    # $((...)) to calculate mathematical expressions
    sum=$(($1+$2))
    echo -n $sum
}

: 1

In [24]:
add 1 2

3

: 1

#### 1. Capture Return Values via Command Substition Using $()

In [25]:
result=$(add 1 2)
echo $result

3[?2004h


: 1

#### 2. Use Global Variables to Provide Return Values
The _sum_ variable in our add function is actually a global variable.

In [26]:
echo -n $sum

3

: 1

### Variable Scope
Using local variables in order to not pollute the global namespace.

In [27]:
function add_local {
    local sum=$(($1+$2))
    echo -n $sum
}

: 1

In [28]:
add_local 5 6

11

: 1

The global variable from above did not change.

In [29]:
echo -n $sum

3

: 1

#### 3. Argument References
As of Bash 4.3+, we can pass an input argument by reference.

In [30]:
function add_by_ref {
    declare -n add_ref=$3
    add_ref=$(($1+$2)) 
}

: 1

In [31]:
add_by_ref 1 2 myAddRefResult

: 1

In [32]:
echo -n $myAddRefResult

3

: 1

### Sub-shells
1. A sub-shell is a special type of command group that allows us to spawn a new execution environment from the current shell.
2. Instead of curly braces, we use parentheses to delimit the function body.

In [33]:
function add_subshell (
    # 'sum' is a global variable
    sum=$(($1+$2))
    echo -n $sum 
)

: 1

In [34]:
add_subshell 2 3

5

: 1

Our global variable did not change because the function was executed in a sub-shell.

In [35]:
echo -n $sum

3

: 1

### Recursion
Recursion is possible but not typical for shell programming.

In [36]:
function factorial {
    if [ $1 -le 1 ]; then
        echo -n 1
    else
        echo -n $(($(factorial $(($1-1)))*$1))
    fi 
}

: 1

In [37]:
factorial 5

120

: 1

In [38]:
factorial 10

3628800

: 1

# Arrays

## Declaring Arrays
There are two ways:<br>
1. Declare a variable explicitly to be an array.
2. Create an array on the fly.

#### Declaring an array explicitly

In [49]:
declare -p pioneers

bash: declare: pioneers: not found


: 1

In [50]:
declare -a pioneers

: 1

In [51]:
declare -p pioneers

declare -a pioneers


: 1

#### Creating an Array on the Fly
An array is automatically created when you assign a value to a variable.

In [52]:
pioneers[0]='Ken Thompson'

: 1

In [53]:
declare -p pioneers

declare -a pioneers=([0]="Ken Thompson")


: 1

You can assign multiple values at once.

In [55]:
unset pioneers

: 1

In [56]:
pioneers=('Ken Thompson' 'Brian Kernighan' 'Dennis Ritchie')

: 1

In [57]:
declare -p pioneers

declare -a pioneers=([0]="Ken Thompson" [1]="Brian Kernighan" [2]="Dennis Ritchie")


: 1

#### Assigning Values to an Array
1. An array doesn't need to have continuous indexes.
2. When you try to print an array element that is not initialized, you'll get a null value.

In [58]:
pioneers[20]="Douglas McIlroy"

: 1

In [59]:
declare -p pioneers

declare -a pioneers=([0]="Ken Thompson" [1]="Brian Kernighan" [2]="Dennis Ritchie" [20]="Douglas McIlroy")


: 1

### Accessing Array Elements

The default index is zero.

In [60]:
echo $pioneers

Ken Thompson


: 1

This does not work as one might expect coming from a mainstream programming language.

In [61]:
echo $pioneers[1]

Ken Thompson[1]


: 1

This is the correct way.

In [62]:
echo ${pioneers[1]}

Brian Kernighan


: 1

Now the whole array:

In [65]:
echo ${pioneers[@]}

Ken Thompson Brian Kernighan Dennis Ritchie Douglas McIlroy


: 1

In [66]:
echo ${pioneers[*]}

Ken Thompson Brian Kernighan Dennis Ritchie Douglas McIlroy


: 1

We"ll see the difference of the two commands above when we cover looping.

### Appending an Array Element

In [67]:
unset pioneers

: 1

You can also explicitly specify the index of the array entry.

In [68]:
pioneers=('Ken Thompson' 'Brian Kernighan' 'Dennis Ritchie' [20]="Douglas McIlroy")

: 1

In [69]:
declare -p pioneers

declare -a pioneers=([0]="Ken Thompson" [1]="Brian Kernighan" [2]="Dennis Ritchie" [20]="Douglas McIlroy")


: 1

This workaround of appending array elements works only when indices are consecutive.

In [70]:
pioneers=("${pioneers[@]}" 'Joe Ossanna' 'Linus Torvalds')

: 1

In [71]:
declare -p pioneers

declare -a pioneers=([0]="Ken Thompson" [1]="Brian Kernighan" [2]="Dennis Ritchie" [3]="Douglas McIlroy" [4]="Joe Ossanna" [5]="Linus Torvalds")


: 1

The quotes are essential!

In [72]:
pioneers=(${pioneers[@]} 'Richard Stallman')

: 1

In [73]:
declare -p pioneers

declare -a pioneers=([0]="Ken" [1]="Thompson" [2]="Brian" [3]="Kernighan" [4]="Dennis" [5]="Ritchie" [6]="Douglas" [7]="McIlroy" [8]="Joe" [9]="Ossanna" [10]="Linus" [11]="Torvalds" [12]="Richard Stallman")


: 1

### Removing Array Elements

In [75]:
unset pioneers

: 1

In [76]:
pioneers=('Ken Thompson' 'Brian Kernighan' 'Dennis Ritchie' [20]="Douglas McIlroy")

: 1

Bye, bye, Brian!

In [77]:
unset pioneers[1]

: 1

In [79]:
declare -p pioneers

declare -a pioneers=([0]="Ken Thompson" [2]="Dennis Ritchie" [20]="Douglas McIlroy")


: 1

### Array Length

In [80]:
echo -n ${#pioneers[@]}

3

: 1

In [81]:
echo -n ${#pioneers[*]}

3

: 1

In [82]:
echo -n 'Length of the third element:' ${#pioneers[2]}

Length of the third element: 14

: 1

### Looping through an Array

In [87]:
i=1
for item in "${pioneers[@]}"; do
    echo $((i++)). $item
done

1. Ken Thompsonl
2. Dennis Ritchie
3. Douglas McIlroy


: 1

In [88]:
i=1
for item in "${pioneers[*]}"; do
    echo $((i++)). $item
done

1. Ken Thompson Dennis Ritchie Douglas McIlroy


: 1

#### Using File Globbing to Initialize an Array

In [89]:
files=(/etc/[abcdefg]*.conf)

: 1

In [90]:
declare -p files

declare -a files=([0]="/etc/ca-certificates.conf" [1]="/etc/dhcpcd.conf" [2]="/etc/dracut.conf" [3]="/etc/e2scrub.conf" [4]="/etc/gai.conf")


: 1

In [92]:
i=1
for file in ${files[@]}; do
    echo $((i++)). $file
done

1. /etc/ca-certificates.conf004l
2. /etc/dhcpcd.conf
3. /etc/dracut.conf
4. /etc/e2scrub.conf
5. /etc/gai.conf


: 1

#### Loading File Content into an Array

In [93]:
unset pioneers

: 1

In [94]:
cat pioneers.txt

Ken2004l
Dennis
Brian
Douglas
Joe


: 1

In [95]:
pioneers=( $(cat pioneers.txt) )

: 1

In [96]:
declare -p pioneers

declare -a pioneers=([0]="Ken" [1]="Dennis" [2]="Brian" [3]="Douglas" [4]="Joe")


: 1

In [98]:
for pioneer in ${pioneers[@]}; do
    echo $pioneer
done

Ken2004h
Dennis
Brian
Douglas
Joe


: 1